**OLSQ DAC'22 tutorial**

It is recommended to pull from [our GitHub repo](https://github.com/UCLA-VAST/OLSQ) becasue it is sometimes more up-to-date, and there are more branches containing research progress.
However, we upload OLSQ to Python Package Index (`pip`) as well, which we use here.

In [1]:
!pip install OLSQ

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54.5 MB 71.8 MB/s 
  Created wheel for OLSQ: filename=olsq-0.0.4.1-py3-none-any.whl size=32607 sha256=1c54d923385aa927a048bd5a5472f51521f663e7ffc46ae45257e3acd81a3e6a
  Stored in directory: /root/.cache/pip/wheels/03/d3/3d/5f89ef72ad6cfe317f434347f58427ad6933a0f24f1ce6ce2e
Successfully built OLSQ


The name of the package is `OLSQ`, the name of the module is `olsq`, and the name of the main solver class is `OLSQ`.
Additionally, we need to import a class `qcdevice` to load to hardware information.

In [2]:
from olsq import OLSQ 

from olsq.device import qcdevice

We initiate the solver with an **objective** and a **mode**.
The objective can be `depth`, `swap`, or `fidelity`.
The mode can be `normal` or `transition`.

In [3]:
solver = OLSQ("depth", "normal")

We construct an object with all the hardware information.
The arguments are: name, number of qubits `nqubits`, list of edges between the qubits `connection`, duration of a SWAP gate `swap_duration`.
In our case, the QPU has five qubits and a 'bowtie' connectivity.
We assume a SWAP gate is decomposed to three CNOTs.

In [4]:
bowtie_connections = [(0,1), (0,2), (1,2), (2,3), (2,4), (3,4)]
bowtie = qcdevice("bowtie", nqubits=5, connection=bowtie_connections, 
                  swap_duration=3)

We load the hardware information.

In [5]:
solver.setdevice(bowtie)

In [6]:
gate_qubits = [(0,), (1,), (3,), (2,3), (0,), (1,), (2,), (3,), (0,1), (2,3),
               (3,0), (1,2), (0,1), (2,3), (0,), (1,), (2,), (3,), (0,1), (2,3),
               (3,), (3,0), (3,)]
gate_name = ["x", "x", "h", "cnot", "t", "t", "t", "tdg", "cnot", "cnot", 
             "cnot", "cnot", "cnot", "cnot", "tdg", "tdg", "tdg", "t", "cnot",
             "cnot", "s", "cnot", "h"]

In [8]:
for i in range(len(gate_qubits)):
    if len(gate_qubits[i]) == 1:
        print(f"g{i} {gate_name[i]} q{gate_qubits[i][0]}")
    else:
        print(f"g{i} {gate_name[i]} q{gate_qubits[i][0]} q{gate_qubits[i][1]}")

g0 x q0
g1 x q1
g2 h q3
g3 cnot q2 q3
g4 t q0
g5 t q1
g6 t q2
g7 tdg q3
g8 cnot q0 q1
g9 cnot q2 q3
g10 cnot q3 q0
g11 cnot q1 q2
g12 cnot q0 q1
g13 cnot q2 q3
g14 tdg q0
g15 tdg q1
g16 tdg q2
g17 t q3
g18 cnot q0 q1
g19 cnot q2 q3
g20 s q3
g21 cnot q3 q0
g22 h q3


In [9]:
solver.setprogram([4, gate_qubits, gate_name], input_mode="IR")

In [10]:
results = solver.solve()

Trying maximal depth = 11...
Trying maximal depth = 14...
Trying maximal depth = 18...
Compilation time = 0:00:30.299127.
SWAP on physical edge (0,2) at time 7
SWAP on physical edge (3,4) at time 7
Gate 0: x 0 on qubit 0 at time 0
Gate 1: x 1 on qubit 2 at time 0
Gate 2: h 3 on qubit 3 at time 0
Gate 3: cnot 2, 3 on qubits 4 and 3 at time 1
Gate 4: t 0 on qubit 0 at time 1
Gate 5: t 1 on qubit 2 at time 1
Gate 6: t 2 on qubit 4 at time 2
Gate 7: tdg 3 on qubit 3 at time 2
Gate 8: cnot 0, 1 on qubits 0 and 2 at time 2
Gate 9: cnot 2, 3 on qubits 4 and 3 at time 3
Gate 10: cnot 3, 0 on qubits 4 and 2 at time 8
Gate 11: cnot 1, 2 on qubits 2 and 4 at time 4
Gate 12: cnot 0, 1 on qubits 2 and 0 at time 9
Gate 13: cnot 2, 3 on qubits 3 and 4 at time 9
Gate 14: tdg 0 on qubit 2 at time 10
Gate 15: tdg 1 on qubit 0 at time 10
Gate 16: tdg 2 on qubit 3 at time 10
Gate 17: t 3 on qubit 4 at time 10
Gate 18: cnot 0, 1 on qubits 2 and 0 at time 11
Gate 19: cnot 2, 3 on qubits 3 and 4 at time 11
G

In [11]:
solver_swap = OLSQ("swap", "normal")
solver_swap.setdevice(bowtie)
solver_swap.setprogram([4, gate_qubits, gate_name], input_mode="IR")
results_swap = solver_swap.solve()

Trying maximal depth = 11...
Trying maximal depth = 14...
Trying maximal depth = 18...
Compilation time = 0:00:33.735602.
SWAP on physical edge (2,4) at time 7
Gate 0: x 0 on qubit 1 at time 0
Gate 1: x 1 on qubit 0 at time 0
Gate 2: h 3 on qubit 4 at time 0
Gate 3: cnot 2, 3 on qubits 2 and 4 at time 1
Gate 4: t 0 on qubit 1 at time 2
Gate 5: t 1 on qubit 0 at time 2
Gate 6: t 2 on qubit 2 at time 2
Gate 7: tdg 3 on qubit 4 at time 2
Gate 8: cnot 0, 1 on qubits 1 and 0 at time 3
Gate 9: cnot 2, 3 on qubits 2 and 4 at time 3
Gate 10: cnot 3, 0 on qubits 2 and 1 at time 8
Gate 11: cnot 1, 2 on qubits 0 and 2 at time 4
Gate 12: cnot 0, 1 on qubits 1 and 0 at time 9
Gate 13: cnot 2, 3 on qubits 4 and 2 at time 9
Gate 14: tdg 0 on qubit 1 at time 10
Gate 15: tdg 1 on qubit 0 at time 10
Gate 16: tdg 2 on qubit 4 at time 10
Gate 17: t 3 on qubit 2 at time 10
Gate 18: cnot 0, 1 on qubits 1 and 0 at time 11
Gate 19: cnot 2, 3 on qubits 4 and 2 at time 11
Gate 20: s 3 on qubit 2 at time 12
Gate

In [12]:
solver_transition = OLSQ("swap", "transition")
solver_transition.setdevice(bowtie)
solver_transition.setprogram([4, gate_qubits, gate_name], input_mode="IR")
results_transition = solver_transition.solve()

Trying maximal depth = 1...
Trying maximal depth = 2...
Compilation time = 0:00:00.327550.
SWAP on physical edge (2,4) at time 0
Gate 0: x 0 on qubit 0 at time 0
Gate 1: x 1 on qubit 1 at time 0
Gate 2: h 3 on qubit 4 at time 0
Gate 3: cnot 2, 3 on qubits 2 and 4 at time 0
Gate 4: t 0 on qubit 0 at time 0
Gate 5: t 1 on qubit 1 at time 0
Gate 6: t 2 on qubit 2 at time 0
Gate 7: tdg 3 on qubit 4 at time 0
Gate 8: cnot 0, 1 on qubits 0 and 1 at time 0
Gate 9: cnot 2, 3 on qubits 2 and 4 at time 0
Gate 10: cnot 3, 0 on qubits 2 and 0 at time 1
Gate 11: cnot 1, 2 on qubits 1 and 2 at time 0
Gate 12: cnot 0, 1 on qubits 0 and 1 at time 1
Gate 13: cnot 2, 3 on qubits 4 and 2 at time 1
Gate 14: tdg 0 on qubit 0 at time 1
Gate 15: tdg 1 on qubit 1 at time 1
Gate 16: tdg 2 on qubit 4 at time 1
Gate 17: t 3 on qubit 2 at time 1
Gate 18: cnot 0, 1 on qubits 0 and 1 at time 1
Gate 19: cnot 2, 3 on qubits 4 and 2 at time 1
Gate 20: s 3 on qubit 2 at time 1
Gate 21: cnot 3, 0 on qubits 2 and 0 at ti